In [1]:
!pip install dash --upgrade
!pip install jupyter-dash
!pip install pandas plotly


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Import necessary libraries
import dash
import dash.html as html
import dash.dcc as dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Initialize the app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Load the dataset (update the path if needed)
try:
    spacex_df = pd.read_csv("C:/Users/ganes/Downloads/spacex_launch_dash.csv")
except FileNotFoundError:
    print("Error: 'spacex_launch_dash.csv' not found in the current directory.")
    print("Please ensure the file is in the working directory or provide the correct path.")
    # Fallback: Create a dummy dataset for testing
    data = {
        'Launch Site': ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS LC-40'],
        'Payload Mass (kg)': [500, 1000, 2000, 3000],
        'class': [1, 0, 1, 1],
        'Booster Version Category': ['v1.0', 'v1.1', 'FT', 'v1.0']
    }
    spacex_df = pd.DataFrame(data)

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create dropdown options
unique_launch_sites = spacex_df['Launch Site'].unique().tolist()
launch_sites = [{'label': 'All Sites', 'value': 'All Sites'}]
for launch_site in unique_launch_sites:
    launch_sites.append({'label': launch_site, 'value': launch_site})

# Define the layout
app.layout = html.Div(children=[
    html.Div([
        html.H1('SpaceX Launch Records Dashboard',
                style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    ]),
    
    html.Div([
        dcc.Dropdown(
            id='site-dropdown',
            options=launch_sites,
            placeholder='Select a Launch Site here',
            searchable=True,
            clearable=False,
            value='All Sites'
        ),
        html.Div(dcc.Graph(id='success-pie-chart')),
    ], style={'padding': '0 30px'}),

    html.Div([
        html.Div("Payload range (Kg):",
                 style={'color': '#503D36', 'font-size': 20, 'padding': '0 30px', 'margin-left': '11px'}),
        html.Div([
            dcc.RangeSlider(
                id='payload_slider',
                min=0,
                max=10000,
                step=1000,
                marks={
                    0: {'label': '0 Kg', 'style': {'color': '#77b0b1'}},
                    1000: {'label': '1000 Kg'},
                    2000: {'label': '2000 Kg'},
                    3000: {'label': '3000 Kg'},
                    4000: {'label': '4000 Kg'},
                    5000: {'label': '5000 Kg'},
                    6000: {'label': '6000 Kg'},
                    7000: {'label': '7000 Kg'},
                    8000: {'label': '8000 Kg'},
                    9000: {'label': '9000 Kg'},
                    10000: {'label': '10000 Kg', 'style': {'color': '#f50'}},
                },
                value=[min_payload, max_payload]
            ),
        ], style={'padding': '40px 30px'}),
        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ]),
], style={'padding': '0 20px'})

# Callback for the pie chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)
def update_piegraph(site_dropdown):
    if site_dropdown == 'All Sites' or site_dropdown == 'None':
        all_sites = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(
            all_sites,
            names='Launch Site',
            title='Total Success Launches by All Sites',
            hole=.2,
            color_discrete_sequence=px.colors.sequential.RdBu
        )
    else:
        site_specific = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(
            site_specific,
            names='class',
            title='Total Success Launches for Site ↠ ' + site_dropdown,
            hole=.2
        )
    return fig

# Callback for the scatter chart
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id="payload_slider", component_property="value")]
)
def update_scattergraph(site_dropdown, payload_slider):
    if site_dropdown == 'All Sites' or site_dropdown == 'None':
        low, high = payload_slider
        all_sites = spacex_df
        inrange = (all_sites['Payload Mass (kg)'] > low) & (all_sites['Payload Mass (kg)'] < high)
        fig = px.scatter(
            all_sites[inrange],
            x="Payload Mass (kg)",
            y="class",
            title='Correlation Between Payload and Success for All Sites',
            color="Booster Version Category",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)']
        )
    else:
        low, high = payload_slider
        site_specific = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        inrange = (site_specific['Payload Mass (kg)'] > low) & (site_specific['Payload Mass (kg)'] < high)
        fig = px.scatter(
            site_specific[inrange],
            x="Payload Mass (kg)",
            y="class",
            title='Correlation Between Payload and Success for Site ↠ ' + site_dropdown,
            color="Booster Version Category",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)']
        )
    return fig

# Run the app inline in Jupyter
app.run(jupyter_mode="inline")